In [1]:
import tensorflow as tf

## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [2]:
# In the tf.keras.layers package, layers are objects. To construct a layer, 
# simply construct the object. Most layers take as a first argument the number 
# of output dimensions / channels
layer = tf.keras.layers.Dense(100)

# The number of input dimensions is often unnecessary, as it can be inferred 
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [3]:
# To use a layer, simply call it..
layer(tf.zeros([10,5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.3176206 , -0.5437175 ,  0.41671723, -0.19965404, -0.23443446,
          0.06947792, -0.39285505, -0.60275126,  0.10368264,  0.22423416],
        [ 0.5527218 ,  0.19850534,  0.22221166, -0.4538672 ,  0.17859966,
          0.0369364 ,  0.37361175,  0.4566093 , -0.21481016,  0.3811409 ],
        [ 0.3969416 , -0.2911437 ,  0.32412547,  0.33001304, -0.5879705 ,
         -0.58878064, -0.41621935, -0.42113733, -0.6260586 , -0.39371634],
        [ 0.06009084, -0.19474012,  0.60784656,  0.2196191 , -0.49482697,
          0.3359902 , -0.346225  ,  0.5689425 , -0.32130527, -0.09042585],
        [ 0.54341453, -0.5475145 , -0.13744417,  0.36641008,  0.1244576 ,
          0.24973983,  0.08227253,  0.04046893, -0.4935089 ,  0.5469554 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.3176206 , -0.5437175 ,  0.41671723, -0.19965404, -0.23443446,
          0.06947792, -0.39285505, -0.60275126,  0.10368264,  0.22423416],
        [ 0.5527218 ,  0.19850534,  0.22221166, -0.4538672 ,  0.17859966,
          0.0369364 ,  0.37361175,  0.4566093 , -0.21481016,  0.3811409 ],
        [ 0.3969416 , -0.2911437 ,  0.32412547,  0.33001304, -0.5879705 ,
         -0.58878064, -0.41621935, -0.42113733, -0.6260586 , -0.39371634],
        [ 0.06009084, -0.19474012,  0.60784656,  0.2196191 , -0.49482697,
          0.3359902 , -0.346225  ,  0.5689425 , -0.32130527, -0.09042585],
        [ 0.54341453, -0.5475145 , -0.13744417,  0.36641008,  0.1244576 ,
          0.24973983,  0.08227253,  0.04046893, -0.4935089 ,  0.5469554 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:
  *  `__init__` , where you can do all input-independent initialization
  * `build`, where you know the shapes of the input tensors and can do the rest of the initialization
  * `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel", 
                                        shape=[int(input_shape[-1]),
                                               self.num_outputs])
        
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = MyDenseLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.trainable_variables)

Instructions for updating:
Please use `layer.add_weight` method instead.
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.33347863,  0.16125464, -0.3248241 , -0.10847467,  0.25180614,
         0.6076277 ,  0.6090947 , -0.04466271, -0.62261474,  0.20403552],
       [-0.4901777 , -0.08771855, -0.5381229 , -0.43522784, -0.28080514,
        -0.44514224,  0.3476777 , -0.15949798, -0.11870861, -0.3621147 ],
       [-0.57005584, -0.4714836 , -0.4875036 , -0.31782538, -0.13821575,
        -0.0200569 , -0.49606463, -0.04187655, -0.610464  ,  0.3008889 ],
       [ 0.00906998,  0.44883555, -0

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!

## Models: composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.

The main class used when creating a layer-like thing which contains other layers is tf.keras.Model. Implementing one is done by inheriting from tf.keras.Model.

In [7]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1,2,3])
print(block(tf.zeros([1,2,3,3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0']


Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential

In [8]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1,1),
                                                    input_shape=(
                                                        None, None, 3)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2, 1, 
                                                     padding='same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1,1)),
                              tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=746, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>